<a href="https://colab.research.google.com/github/correalipe/Betting-Data-Analysis/blob/main/Registration_Funnel_and_Customer_Quality_Before_and_After_Validation_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === 1. Montar Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === 2. Importar bibliotecas ===
!pip install xlsxwriter
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 772.5 kB/s eta 0:00:00


In [ ]:
# Definir o caminho base
base_path = '/content/drive/MyDrive/Task 2 - Kaizen Gaming/'

# Carregar os arquivos
customers = pd.read_csv(base_path + 'Customer.txt', sep=',')
acquisition = pd.read_csv(base_path + 'Acquisition Channel.txt', sep=',')
deposits = pd.read_csv(base_path + 'Deposits.csv', sep=',')
ga = pd.read_csv(base_path + 'Google_Analytics_data_202209.csv', sep=',')
date = pd.read_csv(base_path + 'Date.csv', sep=',')

# Visualizar formas das tabelas para checagem
print("Customers:", customers.shape)
print("Acquisition:", acquisition.shape)
print("Deposits:", deposits.shape)
print("Google Analytics:", ga.shape)
print("Date:", date.shape)

# Mostrar as primeiras linhas de cada tabela para inspeção
customers.head()

/tmp/ipython-input-3-3378825275.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  deposits = pd.read_csv(base_path + 'Deposits.csv', sep=',')


Customers: (1854937, 4)
Acquisition: (1179063, 2)
Deposits: (5891956, 6)
Google Analytics: (3935063, 6)
Date: (92, 22)


,Customer_ID,Customer_DateCreation_ID,Customer_Platform_Name,Customer_Product_Name
0,64755556,20220907,Mobile,Sportsbook
1,64756095,20220907,Mobile,Sportsbook
2,64756977,20220907,Mobile,Sportsbook
3,64757040,20220907,Mobile,Sportsbook
4,64759623,20220907,Mobile,Casino


In [ ]:
# Converter e filtrar datas
customers['Customer_DateCreation_ID'] = pd.to_datetime(customers['Customer_DateCreation_ID'], format='%Y%m%d', errors='coerce')
customers = customers.dropna(subset=['Customer_DateCreation_ID'])
cutoff_date = pd.to_datetime('2022-09-16')
customers['Group'] = customers['Customer_DateCreation_ID'].apply(lambda x: 'before' if x < cutoff_date else 'after')

# IDs válidos na GA
ga_valid = ga[ga['customerId'].notna()].copy()
ga_valid['customerId'] = ga_valid['customerId'].astype(int)
ga_customers = ga_valid[['customerId']].drop_duplicates().rename(columns={'customerId': 'Customer_ID'})

# Merge com grupo
customers_subset = customers[['Customer_ID', 'Group']].drop_duplicates()
email_customers = ga_customers.merge(customers_subset, on='Customer_ID', how='left')
print("Clientes via canal e-mail identificados:", email_customers.shape)

# Deposits + clientes via GA/email
deposits['Transaction_Date'] = pd.to_datetime(deposits['Transaction_Date'], errors='coerce')
deposits_email = deposits.merge(email_customers, left_on='Transaction_Customer_ID', right_on='Customer_ID', how='inner')
print("Deposits após merge com clientes canal e-mail:", deposits_email.shape)

# Agregar indicadores
total_customers = deposits_email.groupby('Group')['Customer_ID'].nunique().rename('Total_Customers')
ftd = deposits_email.groupby('Group')['Customer_ID'].nunique().rename('Customers_With_Deposit')
total_amount = deposits_email.groupby('Group')['Deposit_Amount'].sum().rename('Total_Deposit_Amount (€)')

quality_df = pd.concat([total_customers, ftd, total_amount], axis=1)
quality_df['FTD_Conversion_Rate (%)'] = 100 * quality_df['Customers_With_Deposit'] / quality_df['Total_Customers']
quality_df['Avg_Deposit_per_Customer (€)'] = quality_df['Total_Deposit_Amount (€)'] / quality_df['Customers_With_Deposit']
quality_df = quality_df.reset_index()
quality_df

Clientes via canal e-mail identificados: (668237, 2)
Deposits após merge com clientes canal e-mail: (1891537, 8)


,Group,Total_Customers,Customers_With_Deposit,Total_Deposit_Amount (€),FTD_Conversion_Rate (%),Avg_Deposit_per_Customer (€)
0,after,148115,148115,4.735412e+07,100.0,319.711819
1,before,210603,210603,1.032328e+08,100.0,490.177073


In [ ]:
# Clientes únicos via email com base nos depósitos
total_customers = deposits_email.groupby('Group')['Customer_ID'].nunique().rename('Total_Customers')

# Clientes com pelo menos um depósito
ftd = deposits_email.groupby('Group')['Customer_ID'].nunique().rename('Customers_With_Deposit')

# Total depositado
total_amount = deposits_email.groupby('Group')['Deposit_Amount'].sum().rename('Total_Deposit_Amount (€)')

# Juntar indicadores
quality_df = pd.concat([total_customers, ftd, total_amount], axis=1)
quality_df['FTD_Conversion_Rate (%)'] = 100 * quality_df['Customers_With_Deposit'] / quality_df['Total_Customers']
quality_df['Avg_Deposit_per_Customer (€)'] = quality_df['Total_Deposit_Amount (€)'] / quality_df['Customers_With_Deposit']
quality_df = quality_df.reset_index()
quality_df


,Group,Total_Customers,Customers_With_Deposit,Total_Deposit_Amount (€),FTD_Conversion_Rate (%),Avg_Deposit_per_Customer (€)
0,after,148115,148115,4.735412e+07,100.0,319.711819
1,before,210603,210603,1.032328e+08,100.0,490.177073


In [ ]:
ga['utcDate'] = pd.to_datetime(ga['utcDate'], format='%Y-%m-%d', errors='coerce')

# Agrupar por etapa
step_counts = ga.groupby(['utcDate', 'eventAction'])['fullVisitorId'].nunique().reset_index(name='visits')
step_counts['Group'] = step_counts['utcDate'].apply(lambda x: 'before' if x < pd.to_datetime('2022-09-16') else 'after')

pivot = step_counts.pivot_table(index='Group', columns='eventAction', values='visits', aggfunc='sum').fillna(0)
pivot['conversion_rate'] = pivot['Registration Step 4'] / pivot['Registration Step 1']